# Exploratory Analysis

What are people talking about re: our data?

In [1]:
import pandas as pd
from utils import DTYPE, PARSE_DATES, PROVINCES
import glob

data_paths = glob.glob("../data/processed_data/*.csv")
frames = [pd.read_csv(f,header=0,dtype=DTYPE,parse_dates=PARSE_DATES) for f in data_paths]

total_df = pd.concat(frames, axis=0, ignore_index=True).set_index("id").sort_values("created_at")
total_df["created_at"] = total_df["created_at"].dt.to_period("D").dt.to_timestamp('s')
total_df.head()

/Users/cameron/.local/share/virtualenvs/COVID-Child-Care-Twitter-vC7O4c45/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1091: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


,created_at,screen_name,source,clean_text,original_text,is_retweet,favorite_count,retweet_count,hashtags,urls,mentions,city,province,longitude,latitude
id,,,,,,,,,,,,,,,
1236442738612207620,2020-03-08,Oreobonix,Twitter for Android,"['indian', 'friend', 'dinner', 'white', 'frien...",An Indian friend just had dinner w a white fri...,True,0,0,NaN,NaN,['sairasameerarao'],Mississauga,Ontario,-79.65830,43.57890
1236444031447597056,2020-03-08,QuiDocet_Discit,Twitter for Android,"['indian', 'friend', 'dinner', 'white', 'frien...",An Indian friend just had dinner w a white fri...,True,0,0,NaN,NaN,['sairasameerarao'],Toronto,Ontario,-79.41630,43.70011
1236446314801446912,2020-03-08,Leliye__,Twitter for iPhone,"['break', 'news', 'grove', 'unified', 'school'...",🚨BREAKING NEWS: Elk Grove Unified School Distr...,True,0,0,NaN,NaN,['MarleiMartinez'],NaN,NaN,NaN,NaN
1236447149661073408,2020-03-08,trinachp_19,Twitter for Android,"['indian', 'friend', 'dinner', 'white', 'frien...",An Indian friend just had dinner w a white fri...,True,0,0,NaN,NaN,['sairasameerarao'],London,Ontario,-81.23304,42.98339
1236451877044322305,2020-03-08,JezkaSeesink,Twitter for Android,"['indian', 'friend', 'dinner', 'white', 'frien...",An Indian friend just had dinner w a white fri...,True,0,0,NaN,NaN,['sairasameerarao'],NaN,NaN,-113.64258,60.10867


In [10]:
import plotly.express as px
counts = total_df.groupby(["created_at","province"]).count()["screen_name"].unstack().fillna(1)
# Get grand total for each date
counts = pd.DataFrame(counts.assign(Total=counts.sum(1)).stack()).reset_index().rename({0: "count"},axis=1)

counts = counts[counts["province"].isin(PROVINCES+["Total"])]
idx = pd.MultiIndex.from_product([counts['created_at'].unique(), counts['province'].unique()],names=['created_at', 'province'])
counts = counts.set_index(['created_at', 'province']).reindex(idx).reset_index().sort_values('created_at').fillna(0)
counts
args = {
    "x": "created_at",
    "y": "count",
    "color": "province",
    "line_shape": "spline",
    "title":  "Tweets Over Time",
    "template": "simple_white",
    "labels": {"count": "# Tweets", "created_at": "Date", "province": "Province"},
}

fig = px.line(counts,**args)
# fig.write_html("../visualizations/exploratory_analysis/sample-tweets_over_time.html")
fig.show()